# **RAFT Starter**

## RAFT Models

RAFT has several pretrained models:
 - raft-chairs - trained on FlyingChairs
 - raft-things - trained on FlyingChairs + FlyingThings
 - raft-sintel - trained on FlyingChairs + FlyingThings + Sintel + KITTI
 - raft-kitti - raft-sintel finetuned on only KITTI
 - raft-small - trained on FlyingChairs + FlyingThings

Clone the repo and import libraries

In [ ]:
!git clone https://github.com/princeton-vl/RAFT.git

In [ ]:
!pwd
!cd RAFT
!git pull

In [ ]:
# necessary imports

import os
import sys
import numpy as np
import cv2
import pandas as pd

import torch                     # for all things PyTorch
import torch.nn as nn            # for torch.nn.Module, the parent object for PyTorch models
import torch.nn.functional as F  # for the activation function

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# print gpu info

print(torch.__version__)
print(torch.cuda.get_arch_list())
 
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"Allocated memory: {torch.cuda.memory_allocated(i)/1024**2:.2f} MB")
        print(f"Cached memory: {torch.cuda.memory_reserved(i)/1024**2:.2f} MB")
        print(f"Properties: {torch.cuda.get_device_properties(i)}")
else:
    print("CUDA is not available.")

Add RAFT core to path

In [ ]:
# Add RAFT core to path

sys.path.append('RAFT/core')

Get demo frames

In [ ]:
# get sintel demo frames

demo_path = 'RAFT/demo-frames'
frame1 = cv2.imread(os.path.join(demo_path, 'frame_0020.png'))
frame2 = cv2.imread(os.path.join(demo_path, 'frame_0021.png'))
frame3 = cv2.imread(os.path.join(demo_path, 'frame_0023.png'))

frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
frame3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2RGB)

_, ax = plt.subplots(1, 3, figsize=(15, 8))
ax[0].imshow(frame1)
ax[1].imshow(frame2)
ax[2].imshow(frame3);

## Download models

In [ ]:
%cd RAFT
!./download_models.sh
# !python demo.py --model=models/raft-things.pth --path=demo-frames
%cd ..

### Helper functions

In [ ]:
from collections import OrderedDict
from raft import RAFT
from utils import flow_viz
from utils.utils import InputPadder


# convert to torch and get correct dimensions
def process_img(img, device):
    return torch.from_numpy(img).permute(2, 0, 1).float()[None].to(device)


def load_model(weights_path, args):
    model = RAFT(args)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("current device is")
    print(device)

    try:
        pretrained_weights = torch.load(weights_path, map_location=device)
    except Exception as e:
        raise RuntimeError(f"Fehler beim laden der Gewichte: {e}")

    print(torch.cuda.device_count())

    if torch.cuda.device_count() >= 1:
       model = torch.nn.DataParallel(model)
    
    try:
        model.load_state_dict(pretrained_weights)
    except Exception as e:
        raise RuntimeError(f"Fehler beim setzen der Gewichte: {e}")
    
    model.to(device)

    return model


# perform inference with every model
def inference(model, frame1, frame2, device, pad_mode='sintel', iters=12, flow_init=None, upsample=True, test_mode=True):

    model.eval()
    with torch.no_grad():
        # preprocess
        frame1 = process_img(frame1, device)
        frame2 = process_img(frame2, device)

        # important because raft requires every image to be divisible by 8
        padder = InputPadder(frame1.shape, mode=pad_mode)
        frame1, frame2 = padder.pad(frame1, frame2)

        # predict flow in two different modes
        if test_mode:
          # returns the initial flow (1/8 res) + upsampled flow (upsampled res)
          flow_low, flow_up = model(frame1,
                                    frame2,
                                    iters=iters,
                                    flow_init=flow_init,
                                    upsample=upsample,
                                    test_mode=test_mode)

          return flow_low, flow_up

        else:
            # we get all flow it. for the specified amount of iterations
            flow_iters = model(frame1,
                               frame2,
                               iters=iters,
                               flow_init=flow_init,
                               upsample=upsample,
                               test_mode=test_mode)
            
            return flow_iters


def get_viz(flo):
    flo = flo[0].permute(1,2,0).cpu().numpy()
    return flow_viz.flow_to_image(flo)


# sketchy class to pass to RAFT
class Args():
  def __init__(self, model='', path='', small=False, mixed_precision=True, alternate_corr=False):
    self.model = model
    self.path = path
    self.small = small
    self.mixed_precision = mixed_precision
    self.alternate_corr = alternate_corr

  """ Sketchy hack to pretend to iterate through the class objects """
  def __iter__(self):
    return self

  def __next__(self):
    raise StopIteration

### Load Model

In [ ]:
# load the ".pth" model

model = load_model("RAFT/models/raft-sintel.pth", args=Args())

### Predict Optical Flow

In [ ]:
# run the model with test mode

# flow_low = 1/8 res
# flow_up = full res

flow_low, flow_up = inference(model, frame1, frame2, device='cuda')

In [ ]:
# display output shapes

print("flow_low shape = " + str(flow_low.shape))
print("flow_low shape = " + str(flow_up.shape))

### Display Results

In [ ]:
# display the upsampled flow

flow_low_viz = get_viz(flow_low)
flow_up_viz = get_viz(flow_up)

f, (ax0, ax1) = plt.subplots(1,2, figsize=(15,10))

ax0.imshow(frame1)
ax1.imshow(flow_up_viz)
plt.show()

In [ ]:
# display the low res and upsampled flow

f, (ax0, ax1) = plt.subplots(1,2, figsize=(15,10))

ax0.imshow(flow_low_viz)
ax0.set_title('1/8 res flow')
ax1.imshow(flow_up_viz)
ax1.set_title('convex upsampled flow');

In [ ]:
# run the model with saving all iterations

flow_iters = inference(model, frame1, frame2, device='cuda', iters=20, test_mode=False)

In [ ]:
# visualize first and final flow iteration

f, (ax0, ax1) = plt.subplots(1,2, figsize=(20,10))

ax0.imshow(get_viz(flow_iters[0]))
ax0.set_title('first flow iteration')
ax1.imshow(get_viz(flow_iters[-1]))
ax1.set_title('final flow iteration');

Save flow iters

In [ ]:
for i in range(0, len(flow_iters)):
    # cv2.imwrite(f"flow_iter_{i}.png", get_viz(flow_iters[i]))
    fig = plt.figure(figsize=(20, 10))
    plt.imshow(get_viz(flow_iters[i]))
    plt.title(f"Iteration {i}")
    # fig.savefig(f"flow_iter_{i}.png");

## Flow estimation on a custom pair of frames

In [ ]:
# estimation on a custom pair of frames

demo_path = '/home/max/Dokumente/CV_projectsFork/RAFT/custom_demo_frames'
frame1 = cv2.imread(os.path.join(demo_path, 'm_baseFrameGray.jpg'))
frame2 = cv2.imread(os.path.join(demo_path, 'm_nextFrameGray.jpg'))

frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)

_, ax = plt.subplots(1, 2, figsize=(15, 8))
ax[0].imshow(frame1)
ax[1].imshow(frame2);

## OPTIONAL (use KITTI only model)
# del model
# model = load_model("RAFT/models/raft-kitti.pth", args=Args())
# flow_iters = inference(model, frame1, frame2, device='cuda', pad_mode='kitti', iters=20, test_mode=False)
# flow_iters = inference(model, frame1, frame2, device='cuda', pad_mode='kitti', iters=10, test_mode=False)
flow_iters = inference(model, frame1, frame2, device='cuda', pad_mode='kitti', iters=9, test_mode=False)
# flow_iters = inference(model, frame1, frame2, device='cuda', pad_mode='kitti', iters=5, test_mode=False)

f, (ax0, ax1) = plt.subplots(1,2, figsize=(15,10))

ax0.imshow(get_viz(flow_iters[0]))
ax0.set_title('first flow iteration')
ax1.imshow(get_viz(flow_iters[-1]))
ax1.set_title('final flow iteration');

## Estimate Flow with a Warm Start

In [ ]:
# get previous estimate at 1/8 res
flow_lo, flow_up = inference(model, frame1, frame2, device='cuda', pad_mode=None, iters=20, test_mode=True)

# 0 initialization
flow_lo_cold, flow_up_cold = inference(model, frame2, frame3, device='cuda', pad_mode=None, flow_init=None, iters=20, test_mode=True)

# warm initialization
flow_lo_warm, flow_up_warm = inference(model, frame2, frame3, device='cuda', pad_mode=None, flow_init=flow_lo, iters=20, test_mode=True)

In [ ]:
f, (ax0, ax1) = plt.subplots(1,2, figsize=(20,10))

ax0.imshow(get_viz(flow_up_cold))
ax0.set_title('0 initialized flow')
ax1.imshow(get_viz(flow_up_warm))
ax1.set_title('warm initialized flow');

## Try an example on Kitti

We can either use the KITTI website to download, or select a video sequence from the MOT challenge.

In [ ]:
# !wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_29_drive_0071/2011_09_29_drive_0071_sync.zip
# !jar xf 2011_09_29_drive_0071_sync.zip

!unzip 2011_09_29_drive_0071_sync.zip -d ./

In [ ]:
# !wget https://motchallenge.net/sequenceVideos/KITTI-16-raw.webm

## Compute Flow on a video

In [ ]:
# cap = cv2.VideoCapture("KITTI-16-raw.webm")

# if (cap.isOpened() == False):
#     print("Error opening video file")

# fps = cap.get(cv2.CAP_PROP_FPS)
# raw_frames = []
# flows = []
# flow_lo = None
# i = 0
# while(cap.isOpened()):

#     # read each video frame
#     ret, frame = cap.read()

#     if ret == True:

#         # save to list
#         raw_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

#         # compute flow
#         flow_lo, flow_up = inference(model, frame2, frame3, device='cuda', flow_init=flow_lo, iters=20, test_mode=True)

#         flows.append(flow_up)

#         # increment counter
#         i += 1

#     # Break if nothing is returned
#     else:
#         break

# # clean up
# cap.release()
# cv2.destroyAllWindows()
# del cap

## Flow example on a pair of frames

Let's see how well RAFT does on a single pair of frames from the KITTI driving scene

In [ ]:
# nehme alle bilder

from glob import glob

left_image_paths = sorted(glob('/home/max/Dokumente/CV_projects/RAFT/2011_09_29/2011_09_29_drive_0071_sync/image_02/data/*.png'))
frames = [cv2.imread(path) for path in left_image_paths]

In [ ]:
# zufälliger index

idx = 75
frame1 = frames[idx]
frame2 = frames[idx + 1]
frame3 = frames[idx + 2]

frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
frame3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2RGB)

_, ax = plt.subplots(1, 2, figsize=(20, 8))
ax[0].imshow(frame1)
ax[1].imshow(frame2);

In [ ]:
## OPTIONAL (use KITTI only model)
# del model
# model = load_model("RAFT/models/raft-kitti.pth", args=Args())
flow_iters = inference(model, frame1, frame2, device='cuda', pad_mode='kitti', iters=20, test_mode=False)

In [ ]:
f, (ax0, ax1) = plt.subplots(1,2, figsize=(20,10))

ax0.imshow(get_viz(flow_iters[0]))
ax0.set_title('first flow iteration')
ax1.imshow(get_viz(flow_iters[-1]))
ax1.set_title('final flow iteration');

## Perform warm start flow prediction

In [ ]:
flow_lo, flow_up = inference(model, frame1, frame2, device='cuda', pad_mode='kitti', iters=20, test_mode=True)


# from scratch
flow_lo_cold, flow_up_cold = inference(model, frame2, frame3, device='cuda', pad_mode='kitti', flow_init=None, iters=20, test_mode=True)

# passing 1/8 sampled flow
flow_lo_warm, flow_up_warm = inference(model, frame2, frame3, device='cuda', pad_mode='kitti', flow_init=flow_lo, iters=20, test_mode=True)

In [ ]:
f, (ax0, ax1) = plt.subplots(1,2, figsize=(30,10))

ax0.imshow(get_viz(flow_up_cold))
ax0.set_title('0 initialized flow')
ax1.imshow(get_viz(flow_up_warm))
ax1.set_title('warm initialized flow');

# **Make a GIF of the fan video**

In this section, we will estimate the flow of each frame using a warm start. In the previous estimations there is a lot of noise in the background of the fan. We will need to remove this noise in order to get good estimations.

Get warm-start flow estimates

In [ ]:
# store results
unfiltered_flow_frames = []

for i in range(2, len(frame_paths)):
    # get frames
    frame_1 = cv2.cvtColor(cv2.imread(frame_paths[i-2]), cv2.COLOR_BGR2RGB)
    frame_2 = cv2.cvtColor(cv2.imread(frame_paths[i-1]), cv2.COLOR_BGR2RGB)
    frame_3 = cv2.cvtColor(cv2.imread(frame_paths[i]), cv2.COLOR_BGR2RGB)

    # get previous estimate at 1/8 res
    flow_lo, flow_up = inference(model, frame_1, frame_2, device='cuda', pad_mode=None, iters=20, test_mode=True)

    # 0 initialization
    flow_lo_cold, flow_up_cold = inference(model, frame_2, frame_3, device='cuda', pad_mode=None, flow_init=None, iters=20, test_mode=True)

    # warm initialization
    flow_lo_warm, flow_up_warm = inference(model, frame_2, frame_3, device='cuda', pad_mode=None, flow_init=flow_lo, iters=20, test_mode=True)

    # save results
    if i == 2:
      unfiltered_flow_frames.append(flow_up)

    unfiltered_flow_frames.append(flow_up_warm)

Display results

In [ ]:
idx = 0

warm = unfiltered_flow_frames[idx].clone()
warm[torch.abs(warm) < 2] = 0

og_frame = cv2.cvtColor(cv2.imread(frame_paths[idx]), cv2.COLOR_BGR2RGB)

f, ax = plt.subplots(1, 2, figsize=(20,10))

ax[0].imshow(og_frame)
ax[0].set_title('Original frame')
ax[1].imshow(get_viz(warm))
ax[1].set_title('Optical Flow from Unfiltered frames');

## **Second Attempt with Filtered Frames**

Now we will show how some basic filtering can make a big difference for flow estimation

Code to filter image

In [ ]:
def filter_image(frame, iters=1):

    filtered = frame.copy()

    for i in range(iters):
      filtered = cv2.GaussianBlur(filtered, dst=None, ksize=(3,3), sigmaX=3)
      filtered = cv2.Laplacian(filtered, dst=None, ddepth=-1, ksize=1, scale=7, delta=1)
      filtered = cv2.GaussianBlur(filtered, dst=None, ksize=(5,5), sigmaX=5)
      filtered[filtered < 10] = 0
      filtered = cv2.convertScaleAbs(filtered, dst=None, alpha=10, beta=0)

    return filtered

In [ ]:
# store results
cold_flow_frames = []
flow_frames = []

for i in range(2, len(frame_paths)):
    # get frames
    frame_1 = cv2.cvtColor(cv2.imread(frame_paths[i-2]), cv2.COLOR_BGR2GRAY)
    frame_2 = cv2.cvtColor(cv2.imread(frame_paths[i-1]), cv2.COLOR_BGR2GRAY)
    frame_3 = cv2.cvtColor(cv2.imread(frame_paths[i]), cv2.COLOR_BGR2GRAY)

    # Filter
    frame_1 = filter_image(frame_1, iters=2)
    frame_2 = filter_image(frame_2, iters=2)
    frame_3 = filter_image(frame_3, iters=2)

    # repeat last 3 dimensions
    frame_1 = np.repeat(frame_1[:, :, None], 3, axis=-1)
    frame_2 = np.repeat(frame_2[:, :, None], 3, axis=-1)
    frame_3 = np.repeat(frame_3[:, :, None], 3, axis=-1)

    # get previous estimate at 1/8 res
    flow_lo, flow_up = inference(model, frame_1, frame_2, device='cuda', pad_mode=None, iters=20, test_mode=True)

    # 0 initialization
    flow_lo_cold, flow_up_cold = inference(model, frame_2, frame_3, device='cuda', pad_mode=None, flow_init=None, iters=20, test_mode=True)

    # warm initialization
    flow_lo_warm, flow_up_warm = inference(model, frame_2, frame_3, device='cuda', pad_mode=None, flow_init=flow_lo, iters=20, test_mode=True)

    # save results
    if i == 2:
      flow_frames.append(flow_up)
      cold_flow_frames.append(flow_up)

    flow_frames.append(flow_up_warm)
    cold_flow_frames.append(flow_up_cold)

### Check and filter results

In [ ]:
# filter results

idx = 0
cold = cold_flow_frames[idx].clone()
cold[torch.abs(cold) < 2] = 0

warm = flow_frames[idx].clone()
warm[torch.abs(warm) < 2] = 0

og_frame = cv2.cvtColor(cv2.imread(frame_paths[idx]), cv2.COLOR_BGR2RGB)
frame = cv2.cvtColor(og_frame, cv2.COLOR_RGB2GRAY)
frame = filter_image(frame)
frame = filter_image(frame)

f, ax = plt.subplots(1, 3, figsize=(20,10))

ax[0].imshow(frame)
ax[0].set_title('Filtered frame')
ax[1].imshow(get_viz(cold))
ax[1].set_title('0 initialized flow')
ax[2].imshow(get_viz(warm))
ax[2].set_title('warm initialized flow');

### Display everything

In [ ]:
len(frame_paths)

In [ ]:
len(unfiltered_flow_frames), len(flow_frames)

In [ ]:
idx = 70
thresh = 2

un_flow = unfiltered_flow_frames[idx].clone()
un_flow[torch.abs(un_flow) < thresh] = 0

flow = flow_frames[idx + 1].clone()
flow[torch.abs(flow) < thresh] = 0

og_frame = cv2.cvtColor(cv2.imread(frame_paths[idx]), cv2.COLOR_BGR2RGB)
frame = cv2.cvtColor(og_frame, cv2.COLOR_RGB2GRAY)
frame = filter_image(frame)
frame = filter_image(frame)

fig, ax = plt.subplots(2, 2, figsize=(20, 10))

ax[0, 0].imshow(og_frame)
ax[0, 0].set_title('Original frame')
ax[0, 0].axis(False)

ax[0, 1].imshow(frame)
ax[0, 1].set_title('Filtered frame')
ax[0, 1].axis(False)

ax[1, 0].imshow(get_viz(un_flow))
ax[1, 0].set_title('Unfiltered Optical Flow');
ax[1, 0].axis(False);

ax[1, 1].imshow(get_viz(flow))
ax[1, 1].set_title('Filtered Optical Flow');
ax[1, 1].axis(False);

In [ ]:
from PIL import Image
from glob import glob


def create_gif_from_images(save_path, image_path, ext):
    ''' creates a GIF from a folder of images
        Inputs:
            save_path (str) - path to save GIF
            image_path (str) - path where images are located
            ext (str) - extension of the images
        Outputs:
            None

        Update:
            Add functionality for multiple extensions
    '''
    image_paths = sorted(glob(os.path.join(image_path, f'*.{ext}')))

    pil_images = [Image.open(im_path ) for im_path in image_paths]

    pil_images[0].save(save_path, format='GIF', append_images=pil_images,
                      save_all=True, duration=50, loop=0)

In [ ]:
thresh = 2


for i in range(len(flow_frames) - 1):

  un_flow = unfiltered_flow_frames[i].clone()
  un_flow[torch.abs(un_flow) < thresh] = 0

  flow = flow_frames[i + 1].clone()
  flow[torch.abs(flow) < thresh] = 0

  og_frame = cv2.cvtColor(cv2.imread(frame_paths[i]), cv2.COLOR_BGR2RGB)
  frame = cv2.cvtColor(og_frame, cv2.COLOR_RGB2GRAY)
  frame = filter_image(frame)
  frame = filter_image(frame)

  fig, ax = plt.subplots(2, 2, figsize=(20, 10))

  ax[0, 0].imshow(og_frame)
  ax[0, 0].set_title('Original frame')
  ax[0, 0].axis(False)

  ax[0, 1].imshow(frame)
  ax[0, 1].set_title('Filtered frame')
  ax[0, 1].axis(False)

  ax[1, 0].imshow(get_viz(un_flow))
  ax[1, 0].set_title('Unfiltered Optical Flow');
  ax[1, 0].axis(False);

  ax[1, 1].imshow(get_viz(flow))
  ax[1, 1].set_title('Filtered Optical Flow');
  ax[1, 1].axis(False);

  fig.savefig(f"flow_gif_{i}.jpg");
  plt.close();


In [ ]:
gif_frame_paths = sorted(glob('*.jpg'))
gif_frame_paths.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
gif_path = 'flow.gif'

In [ ]:
pil_images = [Image.open(im_path) for im_path in gif_frame_paths]
pil_images[0].save(gif_path, format='GIF', append_images=pil_images,
                      save_all=True, duration=50, loop=0)